In [1]:
!pip install numpy tensorflow pathlib wget Flask Werkzeug
!pip install flask_cors



Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
from __future__ import division, print_function
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from flask import Flask, request, jsonify
from werkzeug.utils import secure_filename
from flask_cors import CORS 
import imghdr 

MODEL_PATH = '/home/vincent/Desktop/mobile projects/apple disease detector/fruit disease/fruit_disease_model.hdf5' 
UPLOAD_FOLDER = os.path.join(os.path.expanduser("~"), "Desktop", "mobile projects", "images")

if not os.path.isfile(MODEL_PATH):
    print(f"Model file {MODEL_PATH} not found.")
    exit()

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
CORS(app)

def model_predict(img_path):
    print("Loading model from:", MODEL_PATH)
    model = load_model(MODEL_PATH)
    print("Model loaded successfully.")
    img = image.load_img(img_path, target_size=(256, 256)) 
    x = image.img_to_array(img)
    x = x / 255.0 
    x = np.expand_dims(x, axis=0)
    preds = model.predict(x)
    print("Raw Predictions:", preds)
    preds = np.argmax(preds, axis=1)
    print("Argmax Predictions:", preds) 
    return preds

@app.route('/detect', methods=['POST'])
def index():
    if 'image' not in request.files:
        return jsonify({'error': 'No image file uploaded'})

    f = request.files['image']
    if f.filename == '':
        return jsonify({'error': 'No selected file'})
    
    if imghdr.what(f) not in ['jpeg', 'png', 'gif', 'bmp']:
        return jsonify({'error': 'Invalid image file'})

    file_path = os.path.join(app.config['UPLOAD_FOLDER'], secure_filename(f.filename))
    f.save(file_path)
    
    preds = model_predict(file_path)
    pred_class = preds[0]

    if pred_class == 0:
        result = "The fruit has multiple diseases."
        recommendation = "Consider applying appropriate fungicides and pesticides."
    elif pred_class == 1:
        result = "The fruit is healthy."
        recommendation = "Continue regular care and maintenance to keep the fruit healthy."
    elif pred_class == 2:
        result = "The fruit has rust."
        recommendation = "Apply suitable fungicides and remove infected leaves to prevent spreading."
    elif pred_class == 3:
        result = "The fruit has scab."
        recommendation = "Implement proper pruning and sanitation practices to manage the disease."
    else:  
        result = "Opps something went wrong."
        recommendation = "You can try to upload a different image."


    return jsonify({'result': result, 'recommendation': recommendation})

if __name__ == '__main__':
    app.run(port=5080)


2024-04-16 14:14:54.344410: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-16 14:14:54.582117: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-16 14:14:55.570887: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-16 14:14:58.418732: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5080
Press CTRL+C to quit


Loading model from: /home/vincent/Desktop/mobile projects/apple disease detector/fruit disease/fruit_disease_model.hdf5


/home/vincent/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step


127.0.0.1 - - [16/Apr/2024 15:49:19] "POST /detect HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Apr/2024 15:49:19] "POST /detect HTTP/1.1" 200 -


Raw Predictions: [[0.45555717 0.02409495 0.14399196 0.37635598]]
Argmax Predictions: [0]
